In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, fbeta_score
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from imblearn.over_sampling import RandomOverSampler

In [107]:
df = pd.read_csv('train.csv', sep = '\t', encoding='latin')
df.head()

,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
0,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1b23.cif1_R,R.G.1.,1,2.6,74,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
df_test = pd.read_csv('test.csv', encoding='latin')
df_test.head()

,Id,index,pdb_chain,DSSR,xray,resol,chainlen,protein,alpham2,betam2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,0,28,1feu.cif1_F,F.C.93.,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,1,0,0
1,1,29,1feu.cif1_F,F.C.93.,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,0,0,0,1,0
2,2,30,1feu.cif1_F,F.C.93.,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,0,1,0,0,0,0
3,3,31,1feu.cif1_F,F.C.93.,1,2.3,21,1,-73.3,174.9,...,0,0,0,0,1,0,0,0,0,0
4,4,32,1feu.cif1_F,F.C.93.,1,2.3,21,1,-73.3,174.9,...,0,0,1,0,0,0,0,0,0,0


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5370 entries, 0 to 5369
Columns: 386 entries, pdb_chain to mg
dtypes: float64(381), int64(3), object(2)
memory usage: 15.8+ MB


Сначала посмотрим, есть ли у нас признаки, которые совпадают на всех объектах обучающей выборки.

In [110]:
columns = []
for i, column in enumerate(df.columns):
    if len(np.unique(np.array(df[column]))) == 1:
        columns.append(column)
        
print(columns)

['xray']


Удалим этот признак из обоих датасетов. (В тестовой выборке этот признак тоже такой же)

In [111]:
for column in columns:
    df.drop(column, axis=1, inplace=True)
    df_test.drop(column, axis=1, inplace=True)
df.head()

,pdb_chain,DSSR,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
0,1b23.cif1_R,R.G.1.,2.6,74,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1b23.cif1_R,R.G.1.,2.6,74,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1b23.cif1_R,R.G.1.,2.6,74,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1b23.cif1_R,R.G.1.,2.6,74,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1b23.cif1_R,R.G.1.,2.6,74,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Удалим первые 2 колонки в тестовом датасете, так как первая из них это индекс стоки, а второй нет в обучающем датасете.

In [112]:
df_test.drop(['Id', 'index'], axis=1, inplace=True)
df_test.head()

,pdb_chain,DSSR,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,0,0,1,0,0
1,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,0,0,0,1,0
2,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,1,0,0,0,0
3,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,1,0,0,0,0,0
4,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,1,0,0,0,0,0,0,0


Также видим, что в наших данных много NaN. Заменим их на среднее по столбцу.

In [113]:
##удаление их
df.dropna(inplace=True)

In [77]:
#для обучающего
for column in df.columns:
    if df[column].isnull().values.any():
        df[column].fillna('to_replace', inplace=True)
        temp = np.array(df[column])
        mean = np.mean(temp[temp!='to_replace'])
        temp[temp=='to_replace'] = mean
        df[column] = temp
        
#для тестового
for column in df_test.columns:
    if df_test[column].isnull().values.any():
        df_test[column].fillna('to_replace', inplace=True)
        temp = np.array(df_test[column])
        mean = np.mean(temp[temp!='to_replace'])
        temp[temp=='to_replace'] = mean
        df_test[column] = temp

df.head()

,pdb_chain,DSSR,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
0,1b23.cif1_R,R.G.1.,2.6,74,1,-34.2217,56.1031,48.2511,89.1473,-132.533,...,0,0,0,0,0,0,0,0,1,0
1,1b23.cif1_R,R.G.1.,2.6,74,1,-34.2217,56.1031,48.2511,89.1473,-132.533,...,0,0,0,0,0,0,1,0,0,0
2,1b23.cif1_R,R.G.1.,2.6,74,1,-34.2217,56.1031,48.2511,89.1473,-132.533,...,0,0,0,0,0,0,0,1,0,0
3,1b23.cif1_R,R.G.1.,2.6,74,1,-34.2217,56.1031,48.2511,89.1473,-132.533,...,0,0,0,0,1,0,0,0,0,0
4,1b23.cif1_R,R.G.1.,2.6,74,1,-34.2217,56.1031,48.2511,89.1473,-132.533,...,0,0,0,1,0,0,0,0,0,0


In [79]:
df_test.head()

,pdb_chain,DSSR,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,0,0,1,0,0
1,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,0,0,0,1,0
2,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,0,1,0,0,0,0
3,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,0,0,1,0,0,0,0,0
4,1feu.cif1_F,F.C.93.,2.3,21,1,-73.3,174.9,53.7,83.5,-154.8,...,0,0,1,0,0,0,0,0,0,0


Удалим второй признак, так как он неинформативный

In [114]:
df.drop('DSSR', axis=1, inplace=True)
df_test.drop('DSSR', axis=1, inplace=True)
df.head()

,pdb_chain,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,zetam2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
35,1b23.cif1_R,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36,1b23.cif1_R,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
37,1b23.cif1_R,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
38,1b23.cif1_R,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39,1b23.cif1_R,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Видим, что у нас есть 2 категориальных признака (первые 2 признака). Закодируем их используя OneHot encoder

In [115]:
#индексы категориальных признаков
indeces = np.array([0])

#применяем LabelEncoder
int_encoder = LabelEncoder()
for ind in indeces:
    values = np.hstack((np.array(df[df.columns[ind]]), np.array(df_test[df.columns[ind]])))
    int_encoder.fit(values)
    df[df.columns[ind]] = int_encoder.transform(df[df.columns[ind]])
    df_test[df.columns[ind]] = int_encoder.transform(df_test[df.columns[ind]])


X = np.array(df.iloc[:,:-1])
y = np.array(df.iloc[:,-1], dtype=int)
X_test = np.array(df_test)
pd.DataFrame(X).head()
y[y > 0.5] = 1
y[y < 0.5] = 0

#применяем OneHotEncoder
bin_encoder = OneHotEncoder()
values = np.vstack((X[:,[0]], X_test[:,[0]]))
bin_encoder.fit(values)

category_data = bin_encoder.transform(X[:,[0]]).toarray()
non_category_data = X[:, 1:]
X = np.hstack((category_data, non_category_data))

category_data_test = bin_encoder.transform(X_test[:,[0]]).toarray()
non_category_data_test = X_test[:, 1:]
X_test = np.hstack((category_data_test, non_category_data_test))

pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,373,374,375,376,377,378,379,380,381,382
0,0.0,2.6,74.0,1.0,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,2.6,74.0,1.0,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,2.6,74.0,1.0,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,2.6,74.0,1.0,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.6,74.0,1.0,-60.0,177.7,50.4,84.8,-150.6,-79.4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
pd.DataFrame(X_test).head()

,0,1,2,3,4,5,6,7,8,9,...,373,374,375,376,377,378,379,380,381,382
0,12.0,2.3,21.0,1.0,-73.3,174.9,53.7,83.5,-154.8,-71.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,12.0,2.3,21.0,1.0,-73.3,174.9,53.7,83.5,-154.8,-71.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,12.0,2.3,21.0,1.0,-73.3,174.9,53.7,83.5,-154.8,-71.2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,12.0,2.3,21.0,1.0,-73.3,174.9,53.7,83.5,-154.8,-71.2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,12.0,2.3,21.0,1.0,-73.3,174.9,53.7,83.5,-154.8,-71.2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь заметим, что в обущающей выборке классы очень несбалансированы

In [131]:
len(y[y==0]), len(y[y==1])

(3845, 3845)

In [118]:
sampler = RandomOverSampler()
X, y = sampler.fit_sample(X, y)

**PCA**

Попробуем применить PCA, чтоб получить только 100 признаков

In [466]:
pca = PCA(n_components=150, random_state=0)
fit = pca.fit(X, y)
features = fit.transform(X)

Обучаем лес и посмотрим его score на кросс валидации

In [55]:
clf = RandomForestClassifier(n_estimators=1000)
score = cross_val_score(clf, X, y, scoring='f1', n_jobs=-1)
score.mean(), score.min(), score.max()

(0.9394185494137951, 0.9343898573692552, 0.9431416639897205)

CatBoostClassifier

In [23]:
from catboost import CatBoostClassifier

In [27]:
X_eval = X[-100:]
y_eval = y[-100:]
X_train = X[:-100]
y_train = y[:-100]
model = CatBoostClassifier(iterations=100, learning_rate=1, depth=9, loss_function='Logloss', od_type='Iter', od_wait=7, )
fit_model = model.fit(X_train, y_train, eval_set = (X_eval, y_eval), cat_features=[0])



0:	learn: 0.4617512	test: 0.3734848	best: 0.3734848 (0)	total: 198ms	remaining: 19.6s
1:	learn: 0.4223229	test: 0.3717471	best: 0.3717471 (1)	total: 383ms	remaining: 18.8s
2:	learn: 0.3703146	test: 0.2932695	best: 0.2932695 (2)	total: 574ms	remaining: 18.6s
3:	learn: 0.3292073	test: 0.2631491	best: 0.2631491 (3)	total: 754ms	remaining: 18.1s
4:	learn: 0.2879123	test: 0.2308965	best: 0.2308965 (4)	total: 943ms	remaining: 17.9s
5:	learn: 0.2605058	test: 0.1984256	best: 0.1984256 (5)	total: 1.13s	remaining: 17.7s
6:	learn: 0.2370358	test: 0.1825994	best: 0.1825994 (6)	total: 1.31s	remaining: 17.4s
7:	learn: 0.2064470	test: 0.1519830	best: 0.1519830 (7)	total: 1.49s	remaining: 17.1s
8:	learn: 0.1831178	test: 0.1345219	best: 0.1345219 (8)	total: 1.67s	remaining: 16.9s
9:	learn: 0.1743801	test: 0.1357739	best: 0.1345219 (8)	total: 1.86s	remaining: 16.7s
10:	learn: 0.1670916	test: 0.1294289	best: 0.1294289 (10)	total: 2.04s	remaining: 16.5s
11:	learn: 0.1579687	test: 0.1353933	best: 0.1294289

95:	learn: 0.0205123	test: 0.0197089	best: 0.0197089 (95)	total: 18.9s	remaining: 789ms
96:	learn: 0.0201475	test: 0.0191091	best: 0.0191091 (96)	total: 19.1s	remaining: 591ms
97:	learn: 0.0200135	test: 0.0193330	best: 0.0191091 (96)	total: 19.3s	remaining: 393ms
98:	learn: 0.0189090	test: 0.0185938	best: 0.0185938 (98)	total: 19.5s	remaining: 197ms
99:	learn: 0.0163942	test: 0.0160426	best: 0.0160426 (99)	total: 19.6s	remaining: 0us

bestTest = 0.0160425958
bestIteration = 99



**перебор по параметрам**

In [480]:
from catboost import CatBoostClassifier
lrs = [1, 0.9, 0.8,0.7, 0.6]
depths = np.arange(5, 15)
X_eval = features[-100:]
y_eval = y[-100:]
X_train = features[:-100]
y_train = y[:-100]
max_score = 0
for lr in lrs:
    for depth in depths:
        model = CatBoostClassifier(iterations=70, learning_rate=lr, depth=depth, loss_function='Logloss', od_type='Iter', od_wait=7)
        fit_model = model.fit(X_train, y_train, eval_set = (X_eval, y_eval))
        predictions_eval = fit_model.predict(X_eval)
        score = f1_score(y_eval, predictions_eval)
        if max_score < score:
            max_score = score
            params = (lr, depth)
        print('lr: {}, depth: {} --- score: {}'.format(lr, depth, score))

# fit_model = mod

0:	learn: 0.6002145	test: 0.6563076	best: 0.6563076 (0)	total: 32.7ms	remaining: 2.26s
1:	learn: 0.5431428	test: 0.5594729	best: 0.5594729 (1)	total: 64.7ms	remaining: 2.2s
2:	learn: 0.5032616	test: 0.5473425	best: 0.5473425 (2)	total: 91.4ms	remaining: 2.04s
3:	learn: 0.4703057	test: 0.5207616	best: 0.5207616 (3)	total: 124ms	remaining: 2.05s
4:	learn: 0.4464523	test: 0.4996891	best: 0.4996891 (4)	total: 155ms	remaining: 2.01s
5:	learn: 0.4195402	test: 0.4760883	best: 0.4760883 (5)	total: 186ms	remaining: 1.98s
6:	learn: 0.4019941	test: 0.4198383	best: 0.4198383 (6)	total: 212ms	remaining: 1.91s
7:	learn: 0.3785317	test: 0.4169869	best: 0.4169869 (7)	total: 258ms	remaining: 2s
8:	learn: 0.3653891	test: 0.3807553	best: 0.3807553 (8)	total: 374ms	remaining: 2.53s
9:	learn: 0.3478420	test: 0.3651221	best: 0.3651221 (9)	total: 405ms	remaining: 2.43s
10:	learn: 0.3272355	test: 0.3559541	best: 0.3559541 (10)	total: 436ms	remaining: 2.34s
11:	learn: 0.3103297	test: 0.3473984	best: 0.3473984 

58:	learn: 0.0905847	test: 0.2102547	best: 0.2015427 (50)	total: 3.15s	remaining: 587ms
Stopped by overfitting detector  (7 iterations wait)

bestTest = 0.2015426948
bestIteration = 50

lr: 1, depth: 6 --- score: 0.9444444444444444
0:	learn: 0.5800916	test: 0.5575692	best: 0.5575692 (0)	total: 66.6ms	remaining: 4.6s
1:	learn: 0.5065568	test: 0.4867389	best: 0.4867389 (1)	total: 146ms	remaining: 4.97s
2:	learn: 0.4561801	test: 0.4297197	best: 0.4297197 (2)	total: 223ms	remaining: 4.98s
3:	learn: 0.4146549	test: 0.4269459	best: 0.4269459 (3)	total: 299ms	remaining: 4.93s
4:	learn: 0.3759005	test: 0.4380443	best: 0.4269459 (3)	total: 377ms	remaining: 4.91s
5:	learn: 0.3519471	test: 0.4207145	best: 0.4207145 (5)	total: 445ms	remaining: 4.75s
6:	learn: 0.3130875	test: 0.3951154	best: 0.3951154 (6)	total: 529ms	remaining: 4.76s
7:	learn: 0.2872265	test: 0.3504494	best: 0.3504494 (7)	total: 596ms	remaining: 4.62s
8:	learn: 0.2698730	test: 0.3473315	best: 0.3473315 (8)	total: 702ms	remaining: 

27:	learn: 0.0802186	test: 0.2027487	best: 0.1918829 (21)	total: 5.48s	remaining: 8.22s
28:	learn: 0.0784233	test: 0.1906957	best: 0.1906957 (28)	total: 5.69s	remaining: 8.04s
29:	learn: 0.0784227	test: 0.1905617	best: 0.1905617 (29)	total: 5.7s	remaining: 7.6s
30:	learn: 0.0766070	test: 0.2025288	best: 0.1905617 (29)	total: 5.91s	remaining: 7.43s
31:	learn: 0.0761074	test: 0.2025199	best: 0.1905617 (29)	total: 6.11s	remaining: 7.25s
32:	learn: 0.0731310	test: 0.2043611	best: 0.1905617 (29)	total: 6.3s	remaining: 7.07s
33:	learn: 0.0731214	test: 0.2043733	best: 0.1905617 (29)	total: 6.32s	remaining: 6.69s
34:	learn: 0.0691612	test: 0.1895683	best: 0.1895683 (34)	total: 6.52s	remaining: 6.52s
35:	learn: 0.0682913	test: 0.1833923	best: 0.1833923 (35)	total: 6.75s	remaining: 6.38s
36:	learn: 0.0665853	test: 0.1743356	best: 0.1743356 (36)	total: 6.95s	remaining: 6.2s
37:	learn: 0.0664380	test: 0.1712960	best: 0.1712960 (37)	total: 7.17s	remaining: 6.04s
38:	learn: 0.0651110	test: 0.1687277

2:	learn: 0.3029944	test: 0.3980499	best: 0.3980499 (2)	total: 2.58s	remaining: 57.7s
3:	learn: 0.2727554	test: 0.3595515	best: 0.3595515 (3)	total: 3.42s	remaining: 56.4s
4:	learn: 0.2009523	test: 0.2677698	best: 0.2677698 (4)	total: 4.25s	remaining: 55.3s
5:	learn: 0.1850665	test: 0.2409581	best: 0.2409581 (5)	total: 5.11s	remaining: 54.5s
6:	learn: 0.1700630	test: 0.2252593	best: 0.2252593 (6)	total: 5.93s	remaining: 53.3s
7:	learn: 0.1550371	test: 0.2216334	best: 0.2216334 (7)	total: 6.75s	remaining: 52.4s
8:	learn: 0.1447338	test: 0.2052598	best: 0.2052598 (8)	total: 7.58s	remaining: 51.4s
9:	learn: 0.1419536	test: 0.2008897	best: 0.2008897 (9)	total: 8.42s	remaining: 50.5s
10:	learn: 0.1359871	test: 0.2015380	best: 0.2008897 (9)	total: 9.24s	remaining: 49.6s
11:	learn: 0.1298297	test: 0.1921921	best: 0.1921921 (11)	total: 10.1s	remaining: 48.8s
12:	learn: 0.1214024	test: 0.1945715	best: 0.1921921 (11)	total: 10.9s	remaining: 48s
13:	learn: 0.1070521	test: 0.1929447	best: 0.192192

15:	learn: 0.0735554	test: 0.2319947	best: 0.2319947 (15)	total: 57s	remaining: 3m 12s
16:	learn: 0.0538255	test: 0.2237833	best: 0.2237833 (16)	total: 1m	remaining: 3m 9s
17:	learn: 0.0519757	test: 0.2167018	best: 0.2167018 (17)	total: 1m 4s	remaining: 3m 6s
18:	learn: 0.0501387	test: 0.2160216	best: 0.2160216 (18)	total: 1m 8s	remaining: 3m 2s
19:	learn: 0.0479124	test: 0.2175127	best: 0.2160216 (18)	total: 1m 11s	remaining: 2m 59s
20:	learn: 0.0451588	test: 0.2183331	best: 0.2160216 (18)	total: 1m 15s	remaining: 2m 57s
21:	learn: 0.0434883	test: 0.2180892	best: 0.2160216 (18)	total: 1m 19s	remaining: 2m 53s
22:	learn: 0.0408296	test: 0.2089812	best: 0.2089812 (22)	total: 1m 23s	remaining: 2m 50s
23:	learn: 0.0408272	test: 0.2086878	best: 0.2086878 (23)	total: 1m 23s	remaining: 2m 39s
24:	learn: 0.0312771	test: 0.2098204	best: 0.2086878 (23)	total: 1m 26s	remaining: 2m 36s
25:	learn: 0.0306027	test: 0.2018833	best: 0.2018833 (25)	total: 1m 30s	remaining: 2m 33s
26:	learn: 0.0305967	t

...

35:	learn: 0.1601425	test: 0.2353645	best: 0.2353645 (35)	total: 1.73s	remaining: 1.64s
36:	learn: 0.1565790	test: 0.2319363	best: 0.2319363 (36)	total: 1.78s	remaining: 1.59s
37:	learn: 0.1540247	test: 0.2238214	best: 0.2238214 (37)	total: 1.82s	remaining: 1.53s
38:	learn: 0.1491930	test: 0.2245449	best: 0.2238214 (37)	total: 1.87s	remaining: 1.49s
39:	learn: 0.1462905	test: 0.2230423	best: 0.2230423 (39)	total: 1.91s	remaining: 1.43s
40:	learn: 0.1431653	test: 0.2185602	best: 0.2185602 (40)	total: 1.97s	remaining: 1.4s
41:	learn: 0.1393118	test: 0.2066819	best: 0.2066819 (41)	total: 2.01s	remaining: 1.34s
42:	learn: 0.1377997	test: 0.2019448	best: 0.2019448 (42)	total: 2.06s	remaining: 1.29s
43:	learn: 0.1339640	test: 0.1992396	best: 0.1992396 (43)	total: 2.11s	remaining: 1.25s
44:	learn: 0.1313406	test: 0.1986946	best: 0.1986946 (44)	total: 2.15s	remaining: 1.2s
45:	learn: 0.1293128	test: 0.1969281	best: 0.1969281 (45)	total: 2.2s	remaining: 1.15s
46:	learn: 0.1271355	test: 0.194162

32:	learn: 0.1150823	test: 0.2066229	best: 0.2051810 (28)	total: 3.83s	remaining: 4.29s
33:	learn: 0.1102269	test: 0.2031527	best: 0.2031527 (33)	total: 3.95s	remaining: 4.18s
34:	learn: 0.1094981	test: 0.2012426	best: 0.2012426 (34)	total: 4.05s	remaining: 4.05s
35:	learn: 0.1057751	test: 0.2103632	best: 0.2012426 (34)	total: 4.17s	remaining: 3.94s
36:	learn: 0.1035427	test: 0.2072302	best: 0.2012426 (34)	total: 4.28s	remaining: 3.82s
37:	learn: 0.1008700	test: 0.2040970	best: 0.2012426 (34)	total: 4.39s	remaining: 3.7s
38:	learn: 0.0953275	test: 0.2053454	best: 0.2012426 (34)	total: 4.5s	remaining: 3.58s
39:	learn: 0.0925790	test: 0.2031803	best: 0.2012426 (34)	total: 4.61s	remaining: 3.46s
40:	learn: 0.0900601	test: 0.1996400	best: 0.1996400 (40)	total: 4.73s	remaining: 3.35s
41:	learn: 0.0872379	test: 0.1926802	best: 0.1926802 (41)	total: 4.84s	remaining: 3.23s
42:	learn: 0.0856322	test: 0.1936136	best: 0.1926802 (41)	total: 4.95s	remaining: 3.11s
43:	learn: 0.0854641	test: 0.19225

6:	learn: 0.2180132	test: 0.3011093	best: 0.3011093 (6)	total: 2.85s	remaining: 25.6s
7:	learn: 0.2088869	test: 0.2695911	best: 0.2695911 (7)	total: 3.24s	remaining: 25.1s
8:	learn: 0.2021238	test: 0.2537449	best: 0.2537449 (8)	total: 3.62s	remaining: 24.6s
9:	learn: 0.1878222	test: 0.2428875	best: 0.2428875 (9)	total: 4.02s	remaining: 24.1s
10:	learn: 0.1766171	test: 0.2216374	best: 0.2216374 (10)	total: 4.41s	remaining: 23.7s
11:	learn: 0.1685650	test: 0.2225352	best: 0.2216374 (10)	total: 4.8s	remaining: 23.2s
12:	learn: 0.1641513	test: 0.2180585	best: 0.2180585 (12)	total: 5.18s	remaining: 22.7s
13:	learn: 0.1567005	test: 0.2115951	best: 0.2115951 (13)	total: 5.56s	remaining: 22.2s
14:	learn: 0.1481519	test: 0.2033505	best: 0.2033505 (14)	total: 5.95s	remaining: 21.8s
15:	learn: 0.1369857	test: 0.2026031	best: 0.2026031 (15)	total: 6.34s	remaining: 21.4s
16:	learn: 0.1284059	test: 0.2083344	best: 0.2026031 (15)	total: 6.72s	remaining: 21s
17:	learn: 0.1164211	test: 0.2125221	best: 

3:	learn: 0.2649973	test: 0.4006037	best: 0.4006037 (3)	total: 7.36s	remaining: 2m 1s
4:	learn: 0.2419269	test: 0.3395699	best: 0.3395699 (4)	total: 9.15s	remaining: 1m 58s
5:	learn: 0.2067053	test: 0.2864184	best: 0.2864184 (5)	total: 11s	remaining: 1m 57s
6:	learn: 0.1840477	test: 0.2543821	best: 0.2543821 (6)	total: 12.7s	remaining: 1m 54s
7:	learn: 0.1746972	test: 0.2513998	best: 0.2513998 (7)	total: 14.5s	remaining: 1m 52s
8:	learn: 0.1654779	test: 0.2407831	best: 0.2407831 (8)	total: 16.2s	remaining: 1m 49s
9:	learn: 0.1599234	test: 0.2410175	best: 0.2407831 (8)	total: 17.9s	remaining: 1m 47s
10:	learn: 0.1528732	test: 0.2342284	best: 0.2342284 (10)	total: 19.6s	remaining: 1m 45s
11:	learn: 0.1449405	test: 0.2317416	best: 0.2317416 (11)	total: 21.3s	remaining: 1m 42s
12:	learn: 0.1449106	test: 0.2318161	best: 0.2317416 (11)	total: 21.3s	remaining: 1m 33s
13:	learn: 0.1428895	test: 0.2294619	best: 0.2294619 (13)	total: 23s	remaining: 1m 32s
14:	learn: 0.1344627	test: 0.2329726	bes

50:	learn: 0.0302235	test: 0.1449534	best: 0.1449534 (50)	total: 2m 33s	remaining: 57s
51:	learn: 0.0302197	test: 0.1446936	best: 0.1446936 (51)	total: 2m 33s	remaining: 53.1s
52:	learn: 0.0283172	test: 0.1407562	best: 0.1407562 (52)	total: 2m 37s	remaining: 50.5s
53:	learn: 0.0270477	test: 0.1438594	best: 0.1407562 (52)	total: 2m 41s	remaining: 47.9s
54:	learn: 0.0268810	test: 0.1437159	best: 0.1407562 (52)	total: 2m 45s	remaining: 45.2s
55:	learn: 0.0251739	test: 0.1400460	best: 0.1400460 (55)	total: 2m 49s	remaining: 42.4s
56:	learn: 0.0249365	test: 0.1410011	best: 0.1400460 (55)	total: 2m 54s	remaining: 39.7s
57:	learn: 0.0249349	test: 0.1407751	best: 0.1400460 (55)	total: 2m 54s	remaining: 36s
58:	learn: 0.0249334	test: 0.1407042	best: 0.1400460 (55)	total: 2m 54s	remaining: 32.5s
59:	learn: 0.0237758	test: 0.1426582	best: 0.1400460 (55)	total: 2m 58s	remaining: 29.8s
60:	learn: 0.0233619	test: 0.1422824	best: 0.1400460 (55)	total: 3m 3s	remaining: 27.1s
61:	learn: 0.0223971	test:

Смотрим на параметры

In [481]:
max_score, params

(0.9714285714285714, (1, 9))

In [447]:
print(fit_model.get_params())

{'od_type': 'Iter', 'od_wait': 7, 'loss_function': 'Logloss', 'depth': 7, 'learning_rate': 0.6, 'iterations': 70}


In [82]:

predictions_eval = fit_model.predict(X_eval)
f1_score(y_eval, predictions_eval)

0.962962962962963

Засылаем попытку

In [83]:
predictions = fit_model.predict(X_test)

In [84]:
answers1 = pd.read_csv('sample_submission.csv')
answers1['mg'] = np.array(predictions, dtype=int)
answers1.to_csv('attempt1.csv',  index=False)
answers1.head()


,Id,mg
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [85]:
predictions.sum()

712.0

random forest

In [86]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X, y)
pred_4 = clf.predict(X_test)


In [501]:
answers4 = pd.read_csv('sample_submission.csv')
answers4['mg'] = np.array(predictions, dtype=int)
answers4.to_csv('attempt4.csv',  index=False)
answers4.head()

,Id,mg
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


**XGBoost, подбор параметров**

In [43]:
import warnings
warnings.simplefilter("ignore")

In [121]:
lrs = [1, 0.9, 0.8,0.7, 0.6]
depths = np.arange(5, 15)
iters = np.arange(30, 150, 5)
X_eval = X[-100:]
y_eval = y[-100:]
X_train = X[:-100]
y_train = y[:-100]
max_score = 0
for lr in lrs:
    for depth in depths:
        for it in iters:
            model = XGBClassifier(max_depth=depth, learning_rate=lr, n_estimators=it)
            score = cross_val_score(model, X, y, scoring='f1')
#             model.fit(X_train, y_train)
#             pred_eval = model.predict(X_eval)
#             score = f1_score(pred_eval, y_eval)
            if max_score < score.mean():
                max_score = score.mean()
                params = (lr, depth, it)
            print('lr: {}, depth: {}, it: {} --- score: {}'.format(lr, depth, it, score.mean()))

lr: 1, depth: 5, it: 30 --- score: 0.9521307824615105
lr: 1, depth: 5, it: 35 --- score: 0.9543291633641836
lr: 1, depth: 5, it: 40 --- score: 0.9573090695652122
lr: 1, depth: 5, it: 45 --- score: 0.957103874859771
lr: 1, depth: 5, it: 50 --- score: 0.9588428782758106
lr: 1, depth: 5, it: 55 --- score: 0.9590715417414488
lr: 1, depth: 5, it: 60 --- score: 0.959905152800864
lr: 1, depth: 5, it: 65 --- score: 0.9592254742733207
lr: 1, depth: 5, it: 70 --- score: 0.959679998060227
lr: 1, depth: 5, it: 75 --- score: 0.9595639732818925
lr: 1, depth: 5, it: 80 --- score: 0.9594478490459556
lr: 1, depth: 5, it: 85 --- score: 0.9595632881653735
lr: 1, depth: 5, it: 90 --- score: 0.9601298721838588
lr: 1, depth: 5, it: 95 --- score: 0.9603621135836365
lr: 1, depth: 5, it: 100 --- score: 0.9603599885298667
lr: 1, depth: 5, it: 105 --- score: 0.9613903629968684
lr: 1, depth: 5, it: 110 --- score: 0.9612742637851759
lr: 1, depth: 5, it: 115 --- score: 0.9601285540111569
lr: 1, depth: 5, it: 120 --

lr: 1, depth: 11, it: 65 --- score: 0.9588870325937777
lr: 1, depth: 11, it: 70 --- score: 0.9593440691688676
lr: 1, depth: 11, it: 75 --- score: 0.9589996860559565
lr: 1, depth: 11, it: 80 --- score: 0.9593362613676022
lr: 1, depth: 11, it: 85 --- score: 0.9591094679512381
lr: 1, depth: 11, it: 90 --- score: 0.9595654102467565
lr: 1, depth: 11, it: 95 --- score: 0.9593381253094949
lr: 1, depth: 11, it: 100 --- score: 0.9596806345551211
lr: 1, depth: 11, it: 105 --- score: 0.9596806345551211
lr: 1, depth: 11, it: 110 --- score: 0.9599141677970185
lr: 1, depth: 11, it: 115 --- score: 0.959686106502485
lr: 1, depth: 11, it: 120 --- score: 0.9596806345551211
lr: 1, depth: 11, it: 125 --- score: 0.9602514372094092
lr: 1, depth: 11, it: 130 --- score: 0.9599118125391808
lr: 1, depth: 11, it: 135 --- score: 0.9601378386127953
lr: 1, depth: 11, it: 140 --- score: 0.9610468347755092
lr: 1, depth: 11, it: 145 --- score: 0.9608167264053943
lr: 1, depth: 12, it: 30 --- score: 0.9592132039739919
l

KeyboardInterrupt: 

In [117]:
print(max_score, params)

0.9904761904761905 (1, 5, 110)


## Adaboost c удалением nan

In [63]:

X_eval = X[-100:]
y_eval = y[-100:]
X_train = X[:-100]
y_train = y[:-100]
model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7), n_estimators=40, 
                           random_state=0, learning_rate=0.8)
model.fit(X, y)
predictions7 = model.predict(X_test)

In [64]:
predictions7.sum()

352

## xgboost 

In [65]:
model = XGBClassifier(max_depth=9, learning_rate=0.8, n_estimators=40)
model.fit(X, y)
predictions_xg = model.predict(X_test)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [66]:
predictions_xg.sum()

490

In [67]:
res_predictions_10 = np.zeros(len(X_test))
res_predictions_10[predictions7 == 1] = 1
res_predictions_10[predictions_xg == 1] = 1
res_predictions_10.sum()

733.0

In [68]:
answers10 = pd.read_csv('sample_submission.csv')
answers10['mg'] = np.array(res_predictions_10, dtype=int)
answers10.to_csv('attempt10.csv',  index=False)
answers10.head()

,Id,mg
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


## adaboost c предсказаниями вероятностей

In [47]:
X_eval = X[-100:]
y_eval = y[-100:]
X_train = X[:-100]
y_train = y[:-100]
model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7), n_estimators=40, 
                           random_state=0, learning_rate=0.8)
model.fit(X, y)
predictions8 = model.predict_proba(X_test)
# score = f1_score(y_eval, model.predict(X_eval))


In [48]:
predictions8
res_pred_8 = predictions8[:,1]

In [49]:
res_pred_8[res_pred_8>=0.35] = 1
res_pred_8[res_pred_8<0.35] = 0
res_pred_8.sum()

587.0

In [51]:
answers8 = pd.read_csv('sample_submission.csv')
answers8['mg'] = np.array(res_pred_8, dtype=int)
answers8.to_csv('attempt8.csv',  index=False)
answers8.head()

,Id,mg
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


**посылка 5 **- на 0.29437
удаляем nan
+xgboost XGBClassifier(max_depth=9, learning_rate=0.8, n_estimators=40)


## Catboost c удалением nan и только с labelencoding c catfeatures

In [39]:
model = CatBoostClassifier(iterations=180, learning_rate=0.9, depth=9, loss_function='Logloss', od_type='Iter', od_wait=7, )
fit_model = model.fit(X, y, cat_features=[0])
predictions_9 = model.predict(X_test)

0:	learn: 0.4665867	total: 199ms	remaining: 35.6s
1:	learn: 0.3545668	total: 392ms	remaining: 34.9s
2:	learn: 0.3108495	total: 571ms	remaining: 33.7s
3:	learn: 0.2860330	total: 740ms	remaining: 32.6s
4:	learn: 0.2565041	total: 929ms	remaining: 32.5s
5:	learn: 0.2240362	total: 1.11s	remaining: 32.2s
6:	learn: 0.2203625	total: 1.29s	remaining: 32s
7:	learn: 0.1877264	total: 1.49s	remaining: 32s
8:	learn: 0.1816740	total: 1.67s	remaining: 31.6s
9:	learn: 0.1660743	total: 1.85s	remaining: 31.4s
10:	learn: 0.1497205	total: 2.04s	remaining: 31.4s
11:	learn: 0.1465941	total: 2.22s	remaining: 31.1s
12:	learn: 0.1443160	total: 2.41s	remaining: 31s
13:	learn: 0.1312149	total: 2.6s	remaining: 30.8s
14:	learn: 0.1187630	total: 2.79s	remaining: 30.7s
15:	learn: 0.1164855	total: 2.97s	remaining: 30.5s
16:	learn: 0.1129910	total: 3.16s	remaining: 30.3s
17:	learn: 0.1099436	total: 3.35s	remaining: 30.1s
18:	learn: 0.1095976	total: 3.53s	remaining: 29.9s
19:	learn: 0.1080785	total: 3.7s	remaining: 29.6

161:	learn: 0.0054283	total: 30.2s	remaining: 3.35s
162:	learn: 0.0053266	total: 30.4s	remaining: 3.17s
163:	learn: 0.0053017	total: 30.6s	remaining: 2.98s
164:	learn: 0.0051749	total: 30.7s	remaining: 2.79s
165:	learn: 0.0051270	total: 30.9s	remaining: 2.61s
166:	learn: 0.0051061	total: 31.1s	remaining: 2.42s
167:	learn: 0.0050540	total: 31.3s	remaining: 2.23s
168:	learn: 0.0049668	total: 31.5s	remaining: 2.05s
169:	learn: 0.0049608	total: 31.7s	remaining: 1.86s
170:	learn: 0.0049474	total: 31.8s	remaining: 1.68s
171:	learn: 0.0048953	total: 32s	remaining: 1.49s
172:	learn: 0.0047680	total: 32.2s	remaining: 1.3s
173:	learn: 0.0047066	total: 32.4s	remaining: 1.12s
174:	learn: 0.0046927	total: 32.6s	remaining: 931ms
175:	learn: 0.0046766	total: 32.8s	remaining: 745ms
176:	learn: 0.0046582	total: 32.9s	remaining: 558ms
177:	learn: 0.0046512	total: 33.1s	remaining: 372ms
178:	learn: 0.0045295	total: 33.3s	remaining: 186ms
179:	learn: 0.0044483	total: 33.5s	remaining: 0us


In [40]:
predictions_9.sum()

849.0

In [41]:
answers9 = pd.read_csv('sample_submission.csv')
answers9['mg'] = np.array(predictions_9, dtype=int)
answers9.to_csv('attempt9.csv',  index=False)
answers9.head()

,Id,mg
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


## xgboost с удалением nan и xgboost со средним по столбцам

In [69]:
model = XGBClassifier(max_depth=9, learning_rate=0.8, n_estimators=40)
model.fit(X, y)
predictions_nan = model.predict(X_test)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [70]:
predictions_nan.sum()

490

In [85]:
model = XGBClassifier(max_depth=9, learning_rate=0.8, n_estimators=40)
model.fit(X, y)
predictions_mean = model.predict(X_test)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [86]:
predictions_mean.sum()

398

In [87]:
res_predictions_11 = np.zeros(len(X_test))
res_predictions_11[predictions_nan == 1] = 1
res_predictions_11[predictions_mean == 1] = 1
res_predictions_11.sum()

726.0

In [89]:
answers11 = pd.read_csv('sample_submission.csv')
answers11['mg'] = np.array(res_predictions_11, dtype=int)
answers11.to_csv('attempt11.csv',  index=False)
answers11.head()

,Id,mg
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


## xgboost с удалением nan и xgboost со nan c  разными параметрами

In [119]:
model = XGBClassifier(max_depth=9, learning_rate=0.8, n_estimators=40)
model.fit(X, y)
predictions_nan1 = model.predict(X_test)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [120]:
predictions_nan1.sum()

489

In [127]:
model = XGBClassifier(max_depth=10, learning_rate=0.8, n_estimators=30)
model.fit(X, y)
predictions_nan2 = model.predict(X_test)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [128]:
predictions_nan2.sum()

459

In [129]:
res_predictions_12 = np.zeros(len(X_test))
res_predictions_12[predictions_nan1 == 1] = 1
res_predictions_12[predictions_nan2 == 1] = 1
res_predictions_12.sum()

721.0

In [130]:
answers12 = pd.read_csv('sample_submission.csv')
answers12['mg'] = np.array(res_predictions_12, dtype=int)
answers12.to_csv('attempt12.csv',  index=False)
answers12.head()

,Id,mg
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
